In [6]:
!ls -lh

total 453M
-rw-r--r-- 1 jupyter jupyter  31K Nov  1 02:14 classes.csv
-rw-r--r-- 1 jupyter jupyter  34K Oct 31 11:17 Classificator_hachaton.csv
-rw-r--r-- 1 jupyter jupyter  13K Nov  1 02:37 clean.ipynb
-rw-r--r-- 1 jupyter jupyter  22M Nov  1 01:26 cosine_sim
-rw-r--r-- 1 jupyter jupyter 5.9M Nov  1 02:14 df.csv
-rw-r--r-- 1 jupyter jupyter  33M Oct 31 11:00 krugki.zip
-rw-r--r-- 1 jupyter jupyter 102M Oct 31 11:17 MegaRelation_hackaton.csv
-rw-r--r-- 1 jupyter jupyter 8.4M Oct 31 11:00 mero.xlsx
-rw-r--r-- 1 jupyter jupyter 1.1K Oct 31 11:17 mr.sql
-rw-r--r-- 1 jupyter jupyter 153K Oct 31 11:17 org_hackaton.csv
-rw-r--r-- 1 jupyter jupyter  470 Oct 31 11:17 org.sql
-rw-r--r-- 1 jupyter jupyter  67M Nov  1 02:14 pupil.csv
-rw-r--r-- 1 jupyter jupyter  75M Oct 31 11:17 Pupil_hackaton.csv
-rw-r--r-- 1 jupyter jupyter  295 Oct 31 11:17 pupil.sql
-rw-r--r-- 1 jupyter jupyter  86M Nov  1 02:14 relation.csv
-rw-r--r-- 1 jupyter jupyter  46M Oct 31 11:17 request_hackaton.csv
-rw-r--r-- 1 jup

In [1]:
import pandas as pd
import numpy as np
import pickle
import requests, zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from typing import List, Dict
import itertools
from scipy.sparse import csc_matrix
from sklearn.metrics.pairwise import cosine_distances

In [2]:
df_classes = pd.read_csv('Classificator_hachaton.csv', delimiter=';')
df_org = pd.read_csv('org_hackaton.csv', delimiter=';') # Список КЦ # дубликаты одинаковые КЦ с разным метро
df_pupil = pd.read_csv('Pupil_hackaton.csv', delimiter=';') # пользователи КЦ
df_req = pd.read_csv('request_hackaton.csv', delimiter=';')
df_services = pd.read_csv('services_hackaton.csv', delimiter=';')
df_relation = pd.read_csv('MegaRelation_hackaton.csv', delimiter=';')

In [3]:
def classes_preprocess(df_classes: pd.DataFrame) -> pd.DataFrame:
    class_nan = df_classes['id_родительского_классификатора'].isna()
    df_classes.loc[class_nan, 'id_родительского_классификатора'] = df_classes.loc[class_nan, 'id_классификатора'] 
    df_classes.loc[class_nan, 'id_классификатора'] = np.nan
    df_classes['id_родительского_классификатора'] = df_classes['id_родительского_классификатора'].astype(int)
    df_classes = df_classes.sort_values(['id_классификатора'])

    # матчим названия
    id_classes = {
        'Цирковое искусство': 'Искусство цирка',
        'Изобразительное искусство': 'ИЗО',
        'Хореографическое искусство' : 'Хореографическое творчество',
        'Раннее развитие': 'Раннее эстетическое развитие',
    }

    df_classes['Наименование'] = df_classes['Наименование'].replace(id_classes)
    df_classes = df_classes.groupby(['id_родительского_классификатора', 'Наименование']).apply(lambda x: x['id_классификатора'].fillna(method='ffill')).reset_index()
    df_classes = df_classes.drop(['level_2'], axis=1)

    # допущение, что возможная ошибка в id, приравниваем род_классификатор и классификатор
    df_classes.loc[df_classes['id_классификатора'].isna(), 'id_классификатора'] = df_classes.loc[df_classes['id_классификатора'].isna(), 'id_родительского_классификатора']
    df_classes['id_классификатора'] = df_classes['id_классификатора'].astype(int)
    df_classes = df_classes[~df_classes.duplicated()].reset_index(drop=True)
    return df_classes

def pupil_preprocess(df_pupil: pd.DataFrame) -> pd.DataFrame:
    df_pupil.loc[df_pupil['id_ученика'] == 920894, 'возраст'] = abs(df_pupil.loc[df_pupil['id_ученика'] == 920894, 'возраст'])
    df_pupil.loc[df_pupil['id_ученика'] == 920981, 'возраст'] = df_pupil.loc[df_pupil['id_ученика'] == 920981, 'возраст'] + 1000
    df_pupil.loc[df_pupil['id_ученика'] == 931529, 'возраст'] = df_pupil.loc[df_pupil['id_ученика'] == 931529, 'возраст'] + 1000
    df_pupil.loc[df_pupil['id_ученика'] == 938810, 'возраст'] = df_pupil.loc[df_pupil['id_ученика'] == 938810, 'возраст'] + 1000
    df_pupil.loc[df_pupil['id_ученика'] == 2667143, 'возраст'] = abs(df_pupil.loc[df_pupil['id_ученика'] == 2667143, 'возраст'])
    df_pupil.loc[df_pupil['возраст'] > 90, 'возраст'] = 90
    df_pupil['возраст'] = df_pupil['возраст'].fillna(int(df_pupil['возраст'].mean()))
    df_pupil['возраст'] = df_pupil['возраст'].astype(int)
    return df_pupil

def services_preprocess(df_services: pd.DataFrame, df_classes: pd.DataFrame) -> pd.DataFrame:
    df_services['Дата_создания'] = pd.to_datetime(df_services['Дата_создания'])
    df_services = df_services[df_services['Классификатор_услуги'].isin(np.unique(df_classes['id_классификатора']))]
    df_services['длительность_обучения'] = df_services['длительность_обучения'].fillna('1,0000')
    df_services['длительность_обучения'] = df_services['длительность_обучения'].apply(lambda x: int(x.split(',')[0]))
    return df_services
    
def reliation_preprocess(df_relation: pd.DataFrame) -> pd.DataFrame:
    df_relation = df_relation[~df_relation['id_ученика'].isna()]
    df_relation['id_ученика'] = df_relation['id_ученика'].astype(int)
    df_relation['Статус'] = df_relation['Статус'].astype(int)
    df_relation['Дата_создания_записи'] = pd.to_datetime(df_relation['Дата_создания_записи'])
    df_relation['дата_отчисления'] = pd.to_datetime(df_relation['дата_отчисления'])
    return df_relation

def df(df_relation, df_services):
    df = df_relation.loc[:, ['id_ученика', 'id_услуги']].merge(df_services.loc[:, ['id_услуги', 'Классификатор_услуги']],
                                                               on=['id_услуги']).loc[:, ['id_ученика', 'Классификатор_услуги']]
    df['count'] = 1
    df = df.groupby(['id_ученика', 'Классификатор_услуги'])['count'].count().reset_index()
    return df

In [2]:
# df_classes = classes_preprocess(df_classes)
# df_pupil = pupil_preprocess(df_pupil)
# df_services = services_preprocess(df_services, df_classes)
# df_relation = reliation_preprocess(df_relation)
# df = df(df_relation, df_services)

NameError: name 'classes_preprocess' is not defined

In [22]:
# df_classes.to_csv('classes.csv', index=False)
# df_pupil.to_csv('pupil.csv', index=False)
# df_services.to_csv('services.csv', index=False)
# df_relation.to_csv('relation.csv', index=False)
# df.to_csv('df.csv', index=False)

In [2]:
df_classes = pd.read_csv('classes.csv')
df_pupil = pd.read_csv('pupil.csv')
df_services = pd.read_csv('services.csv')
df_relation = pd.read_csv('relation.csv')
df = pd.read_csv('df.csv')

In [178]:
def get_predicts_age(age: int, n: int) -> List[int]:
    class_ids = []
    with open('cosine_sim', 'rb') as f:
        M = pickle.load(f)
    new_user_id = df_pupil.loc[(df_pupil['возраст'] == age), ['id_ученика']].values.reshape(-1, )[0]
    dist = cosine_distances(M, M[new_user_id]).reshape(-1, )
    dist_idx = dist.argsort()
    for classif in dist_idx:
        if len(np.unique(class_ids)) < n:
            try:
                class_ids.append(df.loc[df['id_ученика'] == classif, 'Классификатор_услуги'].values[0])
            except:
                continue
        else:
            break
    seen = set()
    uniq = []
    for x in class_ids:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq

In [188]:
ids = get_predicts_age(1, 5)
ids

[3220710, 3003431, 3220705, 3220406, 3220706]

In [189]:
kids = get_krugki(ids)
kids

[789724, 876891, 1030103, 1098312, 740611]

In [190]:
df_services[df_services['id_услуги'].isin(kids)]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
11167,876891,платно,3003431,10852,Общее,"Театр танца ""Лимонад""",2019-10-09 12:55:55,1,лет
21344,1098312,бесплатно,3220406,29,Индивидуальное,струнные инструменты/ скрипка,2020-08-08 09:58:14,8,лет
29181,1030103,бесплатно,3220705,113,Общее,Кларнет,2020-04-15 16:16:20,8,лет
29799,740611,бесплатно,3220706,35,Общее,уровень I,2019-12-04 17:45:54,3,лет
31743,789724,бесплатно,3220710,152,Общее,уровень I,2019-08-07 12:41:21,3,лет


In [ ]:
def get_top_services(age: int, n: int = 20):
    class_ids = []
    if age > 90:
        age = 90
    elif age < 0:
        age = 0
    new_user_id = df_pupil.loc[(df_pupil['возраст'] == age), ['id_ученика']].values.reshape(-1, )[0]
    dist = cosine_distances(M_service, M_service[new_user_id]).reshape(-1, )
    dist_idx = dist.argsort()
    for classif in dist_idx:
        if len(np.unique(class_ids)) < n:
            try:
                class_ids.append(service_df.loc[service_df['id_ученика'] == classif, 'Классификатор_услуги'].values[0])
            except:
                continue
        else:
            break
    seen = set()
    uniq = []
    for x in class_ids:
        if x not in seen:
            uniq.append(x)  # ['Классификатор_услуги', 'id_организации', 'Тип_расписания']
            seen.add(x)
    services = models.Service.objects.filter(created_at__gt='2016-01-01',
                                             service_class_id__in=uniq).order_by('-created_at').values_list('id',
                                                                                                            'service_class_id',
                                                                                                            'organization_id',
                                                                                                            'schedule_type',
                                                                                                            named=True).distinct()
    service_ids = pd.DataFrame(services).drop_duplicates(['service_class_id', 'organization_id', 'schedule_type'])[
        'id'].values
    return service_ids[:n]


In [166]:
def get_predicts(user_id: int, n: int) -> List[int]:
    class_ids = []
    with open('cosine_sim', 'rb') as f:
        M = pickle.load(f)
    age = df_pupil.loc[df_pupil['id_ученика'] == user_id, 'возраст'].values[0]
    if user_id in df_relation['id_ученика'].values:
        dist = cosine_distances(M, M[user_id]).reshape(-1, )
    else:
        new_user_id = df_pupil.loc[(df_pupil['возраст'] == age), ['id_ученика']].values.reshape(-1, )[0]
        dist = cosine_distances(M, M[new_user_id]).reshape(-1, )
    dist_idx = dist.argsort()
    for classif in dist_idx:
        if len(np.unique(class_ids)) < n:
            try:
                class_ids.append(df.loc[df['id_ученика'] == classif, 'Классификатор_услуги'].values[0])
            except:
                continue
        else:
            break
    seen = set()
    uniq = []
    for x in class_ids:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq

def get_krugki(ids: List[int]) -> List[int]:
    kr = []
    df_services_ = df_services.copy()
    df_services_ = df_services_.loc[~df_services_.duplicated(['Классификатор_услуги', 'id_организации', 'Тип_расписания']), :]
    df_services_ = df_services_.sort_values('Дата_создания', ascending=False)
    for i in ids:
        kr.append(df_services_.loc[(df_services_['Дата_создания'] > '2016-01-01') & (df_services_['Классификатор_услуги'] == i), 'id_услуги'].values.reshape(-1, )[0])
    return kr

In [194]:
%%time
ids = get_predicts(1, 4)
ids

CPU times: user 5.33 s, sys: 35.6 ms, total: 5.36 s
Wall time: 5.36 s


[3220102, 3003038, 3220101, 3003217]

In [195]:
kids = get_krugki(ids)
kids

[1243257, 574257, 1104931, 1167667]

In [196]:
df_services[df_services['id_услуги'].isin(kids)]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
447,574257,платно,3003038,10876,Общее,Яркие краски скрапбукинга,2018-08-16 16:07:35,3,месяцев
3544,1167667,платно,3003217,10898,Общее,"Театр детской песни ""Светлячок"" (2020/2021)",2020-04-09 15:08:48,1,лет
17454,1104931,платно,3220101,10893,Общее,"студия ""Сольфеджио""",2020-08-13 17:56:51,1,лет
18236,1243257,платно,3220102,107,Общее,уровень I,2020-09-25 11:29:02,3,лет


In [193]:
df_services[df_services['id_услуги'].isin(kids)]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
8078,1231080,платно,3003373,10848,Общее,Студия акробатики «ACRO-MIX»,2020-09-22 12:24:17,1,лет
8262,1225449,платно,3003379,10878,Общее,"Студия бальных танцев ""DanceGroup""",2020-09-20 16:11:06,1,лет
16535,1242176,бесплатно,3210300,128,Общее,Живопись,2020-09-24 22:17:37,5,лет
16664,1068517,бесплатно,3210600,23028,Общее,Арт-лаборатория «Куклы-актёры»_20,2020-06-26 11:55:13,1,лет


In [164]:
kids = get_krugki(ids)
kids

[1084600, 921039, 1243257, 1250417]

In [181]:
df_services[df_services['id_услуги'].isin(kids)]

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
447,574257,платно,3003038,10876,Общее,Яркие краски скрапбукинга,2018-08-16 16:07:35,3,месяцев
3544,1167667,платно,3003217,10898,Общее,"Театр детской песни ""Светлячок"" (2020/2021)",2020-04-09 15:08:48,1,лет
16664,1068517,бесплатно,3210600,23028,Общее,Арт-лаборатория «Куклы-актёры»_20,2020-06-26 11:55:13,1,лет
17454,1104931,платно,3220101,10893,Общее,"студия ""Сольфеджио""",2020-08-13 17:56:51,1,лет
18236,1243257,платно,3220102,107,Общее,уровень I,2020-09-25 11:29:02,3,лет
